#### Creating a simple array with random items

In [5]:
import numpy as np 

np.set_printoptions(suppress = True)

# Generatinf random distribution "tensors"
params = np.random.uniform(low = -20, high = 200, size = 15)

In [6]:
# modifing for easy debugging
params[0] = params.max() + 1
params[1] = params.min() - 1
params[2] = 0

In [7]:
params = np.round(params,4)
params

array([194.7485, -20.0944,   0.    ,  17.3086,  11.5532, 114.3453,
        72.5922,  11.1024, 108.8563,  30.072 , 108.287 , -19.0944,
        53.1751,  91.8896,  36.3399])

### Function for clamping

In [8]:
def clamping(params_arr, lower_bound, upper_bound):
    params_arr[params_arr < lower_bound] = lower_bound
    params_arr[params_arr > upper_bound] = upper_bound
    return params_arr

### Function for asymmetric quantization and dequantization

In [13]:
def asymmetric_quantization(params, bits):
    alpha = np.max(params) # the largest value in our "tensor"
    beta = np.min(params) # smallest value in our "tensor"
    scale = (alpha - beta) / (2**bits - 1) # here wa can also use min/max scaler
    zero_point = -1*(beta / scale)
    lower_bound, upper_bound = 0, (2**bits - 1)
    # Quantization 
    quantized = clamping(np.round(params/scale + zero_point), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale, zero_point

def asymmetric_dequantization(params_q, scale, zero_point):
    return scale * (params_q - zero_point)

### Function for symmetric quantization and dequantization

In [22]:
def symmetric_quantization(params, bits):
    alpha = np.max(np.abs(params)) # the max absolute value
    scale = (alpha) / (2**(bits-1) - 1)
    lower_bound, upper_bound = -(2**bits - 1), (2**bits - 1)
    # Quantization 
    quantized = clamping(np.round(params/scale), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale

def symmetric_dequantization(params_q, scale):
    return scale * params_q

### Quantization error

In [23]:
def quantization_error(params, params_q):
    # we can calculate any for of loss here. ** MSE **
    return np.mean((params - params_q)**2)

### Seeing how it plays out

In [24]:
(asymmetric_q, asymmetric_scale, asymmetric_zero) = asymmetric_quantization(params, 8)
(symmetric_q, symmetric_scale) = symmetric_quantization(params, 8)

print(f'Original:')
print(np.round(params, 2))
print('')
print(f'Asymmetric scale: {asymmetric_scale}, zero: {asymmetric_zero}')
print(asymmetric_q)
print('')
print(f'Symmetric scale: {symmetric_scale}')
print(symmetric_q)

Original:
[194.75 -20.09   0.    17.31  11.55 114.35  72.59  11.1  108.86  30.07
 108.29 -19.09  53.18  91.89  36.34]

Asymmetric scale: 0.8425211764705883, zero: 23.850320396903967
[255   0  24  44  38 160 110  37 153  60 152   1  87 133  67]

Symmetric scale: 1.5334527559055118
[127 -13   0  11   8  75  47   7  71  20  71 -12  35  60  24]


In [25]:
# Dequantize the parameters back to 32 bits
params_deq_asymmetric = asymmetric_dequantization(asymmetric_q, asymmetric_scale, asymmetric_zero)
params_deq_symmetric = symmetric_dequantization(symmetric_q, symmetric_scale)

print(f'Original:')
print(np.round(params, 2))
print('')
print(f'Dequantize Asymmetric:')
print(np.round(params_deq_asymmetric,2))
print('')
print(f'Dequantize Symmetric:')
print(np.round(params_deq_symmetric, 2))

Original:
[194.75 -20.09   0.    17.31  11.55 114.35  72.59  11.1  108.86  30.07
 108.29 -19.09  53.18  91.89  36.34]

Dequantize Asymmetric:
[194.75 -20.09   0.13  16.98  11.92 114.71  72.58  11.08 108.81  30.46
 107.97 -19.25  53.2   91.96  36.35]

Dequantize Symmetric:
[194.75 -19.93   0.    16.87  12.27 115.01  72.07  10.73 108.88  30.67
 108.88 -18.4   53.67  92.01  36.8 ]
